#### download  dataset

In [ ]:
#!pip install datasets

In [1]:
from datasets import load_dataset
from PIL import Image
import torch.nn.functional as F
import os
from tqdm.notebook import tqdm
import torch
import numpy as np


In [ ]:

cifar10 = load_dataset('cifar10')


In [ ]:

# View some examples:
image = Image.new('RGB', size=(32*5, 32*2))
for i in range(10):
  im = cifar10['train'][i]['img']
  image.paste(im, ( (i%5)*32, (i//5)*32 ))
image.resize((32*5*4, 32*2*4), Image.NEAREST)

### Utils

In [2]:
def img_to_tensor(im):
    return torch.tensor(np.array(im.convert('RGB'))/255).permute(2, 0, 1).unsqueeze(0) * 2 - 1

def tensor_to_image(t):
    return Image.fromarray(np.array(((t.squeeze().permute(1, 2, 0)+1)/2).clip(0, 1)*255).astype(np.uint8))

def gather(consts: torch.Tensor, t: torch.Tensor):
    """Gather consts for $t$ and reshape to feature map shape"""
    c = consts.gather(-1, t)
    return c.reshape(-1, 1, 1, 1)


### Adding Noise

In [3]:
n_steps = 100 # number of noising steps
beta = torch.linspace(0.0001, 0.04, n_steps)



In [4]:
def add_noise_one_step(xtm1,t):
    mean = gather(1. - beta, t) ** 0.5 * xtm1 # √(1−βt)*xtm1
    var = gather(beta, t) # βt I
    
    eps = torch.randn_like(xtm1) # Noise shaped like xtm1
    return mean + (var ** 0.5) * eps


In [5]:
def add_noise_multi_step(xtm1,n_of_steps):
    x = img_to_tensor(xtm1).squeeze()

    for t in range(n_of_steps):
        t = torch.tensor(t, dtype=torch.long) 
        
        x = add_noise_one_step(x, t)
    return x
        

In [ ]:
n_steps = 100
beta = torch.linspace(0.0001, 0.04, n_steps)
alpha = 1. - beta
alpha_bar = torch.cumprod(alpha, dim=0)


def add_noise_multi_step_fast(x0, t):
    x0 = img_to_tensor(x0).squeeze()
    t = torch.tensor(t, dtype=torch.long)
    
    mean = gather(alpha_bar, t) ** 0.5 * x0 # now alpha_bar
    var = 1-gather(alpha_bar, t) # (1-alpha_bar)
    eps = torch.randn_like(x0)
    return mean + (var ** 0.5) * eps

In [ ]:
# Show im at different stages
im = cifar10['train'][10]['img']
image = Image.new('RGB', size=(32*2, 32))

image.paste(im, ((0, 0)))

#add noise
#im2=add_noise_multi_step(im,50)
im2=add_noise_multi_step_fast(im,40)

im2=tensor_to_image(im2)
            
image.paste(im2, ((32, 0)))


image.resize((32*4*2, 32*4), Image.NEAREST)

# UNet

In [6]:
import unet

In [ ]:
x = torch.randn(10, 3, 32, 32)
t = torch.tensor([50.], dtype=torch.long)

model = unet.UNet()

model_output = model(x, t)
model_output.shape


### Training 

In [ ]:
model = unet.UNet(n_channels=32).cuda()

n_steps = 100
beta = torch.linspace(0.0001, 0.04, n_steps).cuda()
alpha = 1. - beta
alpha_bar = torch.cumprod(alpha, dim=0)


def add_noise_fast(x0, t):
    mean = gather(alpha_bar, t) ** 0.5 * x0
    var = 1-gather(alpha_bar, t)
    eps = torch.randn_like(x0).to(x0.device)
    return mean + (var ** 0.5) * eps, eps 

batch_size = 128 
lr = 2e-4 

losses = [] 

dataset = cifar10['train']#.select(range(100)) 

optim = torch.optim.AdamW(model.parameters(), lr=lr) 

for i in range(0, len(dataset)-batch_size, batch_size):
    
    optim.zero_grad() 
    ims = [dataset[idx]['img'] for idx in range(i,i+batch_size)] 
    tims = [img_to_tensor(im).cuda() for im in ims] 
    x0 = torch.cat(tims) 
    t = torch.randint(0, n_steps, (batch_size,), dtype=torch.long).cuda()  
    xt, noise = add_noise_fast(x0, t) 
    
    pred_noise = model(xt.float(), t) 
    loss = F.mse_loss(noise.float(), pred_noise)
    losses.append(loss.item()) 
    loss.backward() 
    optim.step() 
    if i%200==0:
        print(loss)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(losses)

In [9]:
import hiddenlayer as hl

x = torch.randn(10, 3, 32, 32)
t = torch.tensor([50.], dtype=torch.long)
model = unet.UNet(n_channels=32)

torch.onnx.export(model, (x,t), "./mnist.pb")



#hl.build_graph(model,(x,t))


RuntimeError: Exporting the operator einsum to ONNX opset version 9 is not supported. Support for this operator was added in version 12, try exporting with this version.